In [1]:
import numpy as np
import time
import os
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from random import sample
import csv
from collections import Counter
# Imports from the func.py file
from func import *
import random

In [2]:
# Reads the graph from the graphml file.
G = nx.read_graphml('../data/graph0.graphml', node_type=int)

In [3]:
# Creates an adjacency_list for the graph.
adjacency_list = {}

for node in G.nodes:
    adjacency_list[node] = []

for source, dest in G.edges:
    adjacency_list[source].append(dest)

# Important nodes are computers which are connected to others.
important_nodes = []
for node in adjacency_list:
    if len(adjacency_list[node]) > 0:
        important_nodes.append(node)

In [4]:
# Converts list of tuples into just a list of nodes.
def convertTupleListToList(tuple_list):
    ret = []
    
    for t in tuple_list:
        ret.append(t[1])
        
    return ret

In [5]:
# Generates a ranking of nodes from highest in degree to lowest.
def inDegreeRanking(adjacency_list):
    
    # Counts in degree.
    counter = Counter()
    for node in adjacency_list:
        counter[node] = 0
        for neighbour in adjacency_list[node]:
            counter[neighbour]+=1
    
    # Converts count into list of tuples in format (in degree, node)
    ranking = []
    
    for node in counter:
        ranking.append((counter[node], node))
    
    ranking.sort()
    ranking.reverse()
    
    return convertTupleListToList(ranking)

In [6]:
# Generates a ranking of nodes from highest out degree to lowest.
def outDegreeRanking(adjacency_list):
    
    ranking = []
    for node in adjacency_list:
        ranking.append((len(adjacency_list[node]), node))
    
    ranking.sort()
    ranking.reverse()
        
    return convertTupleListToList(ranking)

In [7]:
# Generates a random ranking of nodes.
def randomRanking(adjacency_list):
    
    ranking = []
    
    for node in adjacency_list:
        ranking.append(node)
    
    random.seed(1)
    random.shuffle(ranking)
    
    return ranking

In [14]:
def localFlowBetweennessRanking(G):

    # Sets up Julia so that we can run Julia files.
    from julia import Julia
    import julia
    # Make sure to change this to your own julia file path.
    julia.install(julia=r"C:\Users\Daniel\AppData\Local\Programs\Julia-1.7.2\bin\julia.exe")
    julia.Julia(runtime=r"C:\Users\Daniel\AppData\Local\Programs\Julia-1.7.2\bin\julia.exe")
    from julia import Main

    # Includes the Julia files.
    Main.include("../julia/local_flow_betweenness.jl");

    perc = .25
    reduced_weight = .1
    
    local50_betweenness = Main.local_flow_betweenness(list(G.nodes()), list(G.edges()), locality_index=.5)
    A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)

    v = A_lf50.A
    counter = Counter()

    ranking = []

    for i in range(len(v)):

        ranking.append((sum(v[i]), i))

    ranking.sort()
    ranking.reverse()

    print(ranking)

    return convertTupleListToList(ranking)

In [15]:
node_ranking = []

node_ranking.append(inDegreeRanking(adjacency_list))
node_ranking.append(outDegreeRanking(adjacency_list))
node_ranking.append(randomRanking(adjacency_list))
node_ranking.append(localFlowBetweennessRanking(G))

[(411.90000000000055, 192), (410.1000000000006, 438), (406.5000000000006, 11), (398.6000000000006, 372), (394.80000000000086, 279), (394.8000000000008, 151), (393.9000000000009, 86), (391.2000000000007, 308), (390.9000000000008, 416), (390.3000000000005, 397), (387.6000000000007, 401), (384.00000000000057, 449), (383.1000000000007, 478), (383.1000000000005, 37), (382.3000000000007, 82), (381.30000000000064, 108), (379.5000000000005, 380), (379.5000000000005, 253), (378.6000000000006, 222), (372.30000000000047, 90), (370.7000000000006, 414), (368.6000000000005, 105), (368.1000000000007, 381), (368.00000000000057, 216), (367.7000000000006, 114), (367.10000000000036, 94), (339.50000000000034, 455), (336.00000000000017, 312), (334.3000000000001, 436), (330.6000000000001, 228), (319.70000000000016, 292), (13.0, 142), (10.299999999999999, 239), (10.1, 426), (9.3, 0), (7.4, 177), (5.1, 245), (3.1, 325), (3.0, 56), (3.0, 19), (2.1, 333), (2.1, 181), (1.0, 500), (1.0, 499), (1.0, 498), (1.0, 49

In [16]:
# Number of different states, we have 501 computers so there are 501 different states.
state_count = len(G.nodes)
# Number of different actions we can take. (for example outgoing edges and incoming edges)
action_count = len(node_ranking)

In [17]:
qtable = np.random.rand(state_count, action_count).tolist()

In [18]:
# Hyperparameters
epochs = 30000
gamma = 0.1
epsilon = 1
decay = 0.1

In [19]:
def infectNode(adjacency_list, vaccinated, infected):
    
    # List of infectable nodes to select our next infection from.
    infectable = []

    # Go through each infected node and find their neighbours to find nodes to spread to.
    for node in infected:
        for neighbour in adjacency_list[node]:
            if neighbour not in vaccinated and neighbour not in infected:
                infectable.append(neighbour)
        
    # If there are no nodes to infect, return False, indicating that no further nodes can be infected.
    if len(infectable) == 0:
        return False
    
    # Pick a new node randomly from the list of infectable nodes and return True indicating a successful infection.
    infected.add(infectable[random.randrange(0, len(infectable))])
    return True

In [20]:
def vaccinateNode(ranking, vaccinated, infected):
        
    for node in ranking:
        if node not in vaccinated:
            vaccinated.add(node)
            if node in infected:
                infected.remove(node)
            return

In [21]:
def step(ranking, adjacency_list, vaccinated, infected):
    
    done = False
    reward = 0
    
    # If there are no more infections return the reward and say that we're done.
    if not infectNode(adjacency_list, vaccinated, infected):
        done = True
        reward = len(adjacency_list)/num_infected
        print(f"{num_infected}/{len(adjacency_list)}")
        
    vaccinateNode(ranking, vaccinated, infected)
    
    return reward, done

In [22]:
infection_history = []

# Training loop
for i in range(epochs):
    # We have to reset the test so we can move onto next epoch.
#     state, reward, done = env.reset()

    state = 0
    reward = 0
    done = False
    infected = set()
    vaccinated = set()
    infection_source = important_nodes[random.randrange(0, len(important_nodes))]
    infected.add(infection_source)
    num_infected = 1

    print("epoch #", i+1, "/", epochs)
    while not done:

        # act randomly sometimes to allow exploration
        if np.random.uniform() < epsilon:
            action = random.randrange(0, len(node_ranking))
        # if not select max action in Qtable (act greedy)
        else:
            action = qtable[state].index(max(qtable[state]))

        # take action
        reward, done = step(node_ranking[action], adjacency_list, vaccinated, infected)
        if not done:
            num_infected+=1
        else:
            print(reward)
        next_state = state + 1
        
        # update qtable value with Bellman equation
#         print(reward + gamma * max(qtable[next_state]))
        
        qtable[state][action] = reward + gamma * max(qtable[next_state])

        # update state
        state = next_state
    
    print(f"Number of nodes infected: {num_infected}")
    infection_history.append(num_infected)
    # The more we learn, the less we take random actions
    epsilon -= decay*epsilon

#     print("\nDone in", steps, "steps".format(steps))

epoch # 1 / 30000
92/501
5.445652173913044
Number of nodes infected: 92
epoch # 2 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 3 / 30000
91/501
5.5054945054945055
Number of nodes infected: 91
epoch # 4 / 30000
86/501
5.825581395348837
Number of nodes infected: 86
epoch # 5 / 30000
87/501
5.758620689655173
Number of nodes infected: 87
epoch # 6 / 30000
121/501
4.140495867768595
Number of nodes infected: 121
epoch # 7 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 8 / 30000
88/501
5.693181818181818
Number of nodes infected: 88
epoch # 9 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 10 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 11 / 30000
9/501
55.666666666666664
Number of nodes infected: 9
epoch # 12 / 30000
111/501
4.513513513513513
Number of nodes infected: 111
epoch # 13 / 30000
81/501
6.185185185185185
Number of nodes infected: 81
epoch # 14 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 15 / 30000
13/501
38.53846153846154
Number of

epoch # 122 / 30000
124/501
4.040322580645161
Number of nodes infected: 124
epoch # 123 / 30000
14/501
35.785714285714285
Number of nodes infected: 14
epoch # 124 / 30000
84/501
5.964285714285714
Number of nodes infected: 84
epoch # 125 / 30000
14/501
35.785714285714285
Number of nodes infected: 14
epoch # 126 / 30000
101/501
4.96039603960396
Number of nodes infected: 101
epoch # 127 / 30000
160/501
3.13125
Number of nodes infected: 160
epoch # 128 / 30000
113/501
4.433628318584071
Number of nodes infected: 113
epoch # 129 / 30000
99/501
5.0606060606060606
Number of nodes infected: 99
epoch # 130 / 30000
56/501
8.946428571428571
Number of nodes infected: 56
epoch # 131 / 30000
150/501
3.34
Number of nodes infected: 150
epoch # 132 / 30000
127/501
3.9448818897637796
Number of nodes infected: 127
epoch # 133 / 30000
15/501
33.4
Number of nodes infected: 15
epoch # 134 / 30000
85/501
5.894117647058824
Number of nodes infected: 85
epoch # 135 / 30000
71/501
7.056338028169014
Number of node

114/501
4.394736842105263
Number of nodes infected: 114
epoch # 243 / 30000
121/501
4.140495867768595
Number of nodes infected: 121
epoch # 244 / 30000
141/501
3.5531914893617023
Number of nodes infected: 141
epoch # 245 / 30000
90/501
5.566666666666666
Number of nodes infected: 90
epoch # 246 / 30000
99/501
5.0606060606060606
Number of nodes infected: 99
epoch # 247 / 30000
15/501
33.4
Number of nodes infected: 15
epoch # 248 / 30000
138/501
3.630434782608696
Number of nodes infected: 138
epoch # 249 / 30000
5/501
100.2
Number of nodes infected: 5
epoch # 250 / 30000
116/501
4.318965517241379
Number of nodes infected: 116
epoch # 251 / 30000
139/501
3.6043165467625897
Number of nodes infected: 139
epoch # 252 / 30000
64/501
7.828125
Number of nodes infected: 64
epoch # 253 / 30000
125/501
4.008
Number of nodes infected: 125
epoch # 254 / 30000
111/501
4.513513513513513
Number of nodes infected: 111
epoch # 255 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 256 / 30000
129/501

119/501
4.2100840336134455
Number of nodes infected: 119
epoch # 404 / 30000
101/501
4.96039603960396
Number of nodes infected: 101
epoch # 405 / 30000
101/501
4.96039603960396
Number of nodes infected: 101
epoch # 406 / 30000
127/501
3.9448818897637796
Number of nodes infected: 127
epoch # 407 / 30000
9/501
55.666666666666664
Number of nodes infected: 9
epoch # 408 / 30000
159/501
3.150943396226415
Number of nodes infected: 159
epoch # 409 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 410 / 30000
116/501
4.318965517241379
Number of nodes infected: 116
epoch # 411 / 30000
88/501
5.693181818181818
Number of nodes infected: 88
epoch # 412 / 30000
14/501
35.785714285714285
Number of nodes infected: 14
epoch # 413 / 30000
110/501
4.554545454545455
Number of nodes infected: 110
epoch # 414 / 30000
113/501
4.433628318584071
Number of nodes infected: 113
epoch # 415 / 30000
146/501
3.4315068493150687
Number of nodes infected: 146
epoch # 416 / 30000
74/501
6.77027027027027
Number of

104/501
4.8173076923076925
Number of nodes infected: 104
epoch # 538 / 30000
20/501
25.05
Number of nodes infected: 20
epoch # 539 / 30000
15/501
33.4
Number of nodes infected: 15
epoch # 540 / 30000
104/501
4.8173076923076925
Number of nodes infected: 104
epoch # 541 / 30000
109/501
4.5963302752293576
Number of nodes infected: 109
epoch # 542 / 30000
75/501
6.68
Number of nodes infected: 75
epoch # 543 / 30000
123/501
4.073170731707317
Number of nodes infected: 123
epoch # 544 / 30000
116/501
4.318965517241379
Number of nodes infected: 116
epoch # 545 / 30000
29/501
17.275862068965516
Number of nodes infected: 29
epoch # 546 / 30000
55/501
9.10909090909091
Number of nodes infected: 55
epoch # 547 / 30000
103/501
4.864077669902913
Number of nodes infected: 103
epoch # 548 / 30000
105/501
4.771428571428571
Number of nodes infected: 105
epoch # 549 / 30000
46/501
10.891304347826088
Number of nodes infected: 46
epoch # 550 / 30000
109/501
4.5963302752293576
Number of nodes infected: 109
e

...

97/501
5.164948453608248
Number of nodes infected: 97
epoch # 29293 / 30000
63/501
7.9523809523809526
Number of nodes infected: 63
epoch # 29294 / 30000
24/501
20.875
Number of nodes infected: 24
epoch # 29295 / 30000
91/501
5.5054945054945055
Number of nodes infected: 91
epoch # 29296 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 29297 / 30000
65/501
7.707692307692308
Number of nodes infected: 65
epoch # 29298 / 30000
95/501
5.273684210526316
Number of nodes infected: 95
epoch # 29299 / 30000
96/501
5.21875
Number of nodes infected: 96
epoch # 29300 / 30000
97/501
5.164948453608248
Number of nodes infected: 97
epoch # 29301 / 30000
8/501
62.625
Number of nodes infected: 8
epoch # 29302 / 30000
89/501
5.629213483146067
Number of nodes infected: 89
epoch # 29303 / 30000
98/501
5.112244897959184
Number of nodes infected: 98
epoch # 29304 / 30000
65/501
7.707692307692308
Number of nodes infected: 65
epoch # 29305 / 30000
101/501
4.96039603960396
Number of nodes infected: 101
epo

77/501
6.5064935064935066
Number of nodes infected: 77
epoch # 29442 / 30000
98/501
5.112244897959184
Number of nodes infected: 98
epoch # 29443 / 30000
53/501
9.452830188679245
Number of nodes infected: 53
epoch # 29444 / 30000
77/501
6.5064935064935066
Number of nodes infected: 77
epoch # 29445 / 30000
95/501
5.273684210526316
Number of nodes infected: 95
epoch # 29446 / 30000
89/501
5.629213483146067
Number of nodes infected: 89
epoch # 29447 / 30000
6/501
83.5
Number of nodes infected: 6
epoch # 29448 / 30000
96/501
5.21875
Number of nodes infected: 96
epoch # 29449 / 30000
12/501
41.75
Number of nodes infected: 12
epoch # 29450 / 30000
88/501
5.693181818181818
Number of nodes infected: 88
epoch # 29451 / 30000
31/501
16.161290322580644
Number of nodes infected: 31
epoch # 29452 / 30000
81/501
6.185185185185185
Number of nodes infected: 81
epoch # 29453 / 30000
92/501
5.445652173913044
Number of nodes infected: 92
epoch # 29454 / 30000
43/501
11.651162790697674
Number of nodes infe

18/501
27.833333333333332
Number of nodes infected: 18
epoch # 29592 / 30000
25/501
20.04
Number of nodes infected: 25
epoch # 29593 / 30000
15/501
33.4
Number of nodes infected: 15
epoch # 29594 / 30000
84/501
5.964285714285714
Number of nodes infected: 84
epoch # 29595 / 30000
12/501
41.75
Number of nodes infected: 12
epoch # 29596 / 30000
90/501
5.566666666666666
Number of nodes infected: 90
epoch # 29597 / 30000
95/501
5.273684210526316
Number of nodes infected: 95
epoch # 29598 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 29599 / 30000
96/501
5.21875
Number of nodes infected: 96
epoch # 29600 / 30000
95/501
5.273684210526316
Number of nodes infected: 95
epoch # 29601 / 30000
13/501
38.53846153846154
Number of nodes infected: 13
epoch # 29602 / 30000
101/501
4.96039603960396
Number of nodes infected: 101
epoch # 29603 / 30000
72/501
6.958333333333333
Number of nodes infected: 72
epoch # 29604 / 30000
83/501
6.036144578313253
Number of nodes infected: 83
epoch # 29605 / 3

98/501
5.112244897959184
Number of nodes infected: 98
epoch # 29739 / 30000
102/501
4.911764705882353
Number of nodes infected: 102
epoch # 29740 / 30000
100/501
5.01
Number of nodes infected: 100
epoch # 29741 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 29742 / 30000
93/501
5.387096774193548
Number of nodes infected: 93
epoch # 29743 / 30000
97/501
5.164948453608248
Number of nodes infected: 97
epoch # 29744 / 30000
82/501
6.109756097560975
Number of nodes infected: 82
epoch # 29745 / 30000
6/501
83.5
Number of nodes infected: 6
epoch # 29746 / 30000
95/501
5.273684210526316
Number of nodes infected: 95
epoch # 29747 / 30000
74/501
6.77027027027027
Number of nodes infected: 74
epoch # 29748 / 30000
50/501
10.02
Number of nodes infected: 50
epoch # 29749 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 29750 / 30000
79/501
6.341772151898734
Number of nodes infected: 79
epoch # 29751 / 30000
42/501
11.928571428571429
Number of nodes infected: 42
epoch # 29752 / 30000


63/501
7.9523809523809526
Number of nodes infected: 63
epoch # 29916 / 30000
81/501
6.185185185185185
Number of nodes infected: 81
epoch # 29917 / 30000
18/501
27.833333333333332
Number of nodes infected: 18
epoch # 29918 / 30000
99/501
5.0606060606060606
Number of nodes infected: 99
epoch # 29919 / 30000
4/501
125.25
Number of nodes infected: 4
epoch # 29920 / 30000
12/501
41.75
Number of nodes infected: 12
epoch # 29921 / 30000
72/501
6.958333333333333
Number of nodes infected: 72
epoch # 29922 / 30000
101/501
4.96039603960396
Number of nodes infected: 101
epoch # 29923 / 30000
100/501
5.01
Number of nodes infected: 100
epoch # 29924 / 30000
84/501
5.964285714285714
Number of nodes infected: 84
epoch # 29925 / 30000
3/501
167.0
Number of nodes infected: 3
epoch # 29926 / 30000
98/501
5.112244897959184
Number of nodes infected: 98
epoch # 29927 / 30000
13/501
38.53846153846154
Number of nodes infected: 13
epoch # 29928 / 30000
12/501
41.75
Number of nodes infected: 12
epoch # 29929 / 

In [33]:
interval = 100

assert len(infection_history) % interval == 0

for i in range(0, len(infection_history), interval):
    start = i
    end = i+interval
    print(sum(infection_history[start:end])/interval)

# print(sum(infection_history[:5000])/5000)
# print(sum(infection_history[5000:10000])/5000)
# print(sum(infection_history[10000:15000])/5000)
# print(sum(infection_history[15000:20000])/5000)
# print(sum(infection_history[20000:25000])/5000)
# print(sum(infection_history[25000:30000])/5000)

78.89
70.26
83.29
74.24
68.5
74.51
66.22
71.81
78.6
76.34
76.35
74.72
76.02
76.27
67.05
81.14
71.03
76.04
68.97
71.72
64.19
68.61
56.43
52.51
50.09
62.92
54.63
61.99
60.08
61.31
57.15
52.92
62.18
61.24
59.48
58.55
59.26
56.5
57.71
60.62
57.88
56.3
57.18
61.28
60.0
62.91
64.89
61.12
64.97
52.08
59.17
57.42
62.45
58.78
62.28
59.26
62.37
60.28
60.52
59.14
59.18
52.96
62.24
58.82
55.41
63.49
54.46
60.68
56.62
61.37
54.31
57.17
59.28
60.83
59.46
58.75
63.06
57.56
58.07
61.24
65.74
62.95
64.68
59.07
57.98
59.69
55.16
60.65
63.58
54.87
67.24
59.74
55.66
57.26
63.13
60.79
63.53
62.2
61.86
56.77
56.72
62.4
65.58
56.08
54.64
60.2
63.59
57.5
63.43
65.3
67.23
63.72
61.36
57.58
57.08
67.11
57.45
63.57
63.6
56.28
57.08
54.91
64.24
68.01
59.24
66.27
66.42
63.3
59.66
63.39
60.02
62.09
58.98
57.8
59.87
58.22
54.77
56.68
53.5
56.71
58.94
56.72
59.64
56.78
63.17
56.63
61.59
60.98
64.68
63.87
67.87
54.77
57.46
55.84
60.06
62.32
53.69
59.88
60.14
60.57
62.43
53.18
64.7
57.11
65.92
60.16
62.02
58.32
58.03
5

In [105]:
print(qtable)

[[5.2698724971527033e-95, 5.2698724971527033e-95, 0.0008350000000000002], [5.269872453751046e-94, 6.958333395958417e-70, 5.269872453751046e-94], [5.269872448979621e-93, 5.2698724490264455e-93, 6.2625058831309996e-77], [5.88313097285383e-82, 5.269872448979617e-92, 5.269872448979617e-92], [5.224361215265558e-91, 5.224361215265558e-91, 1.0020000000000023e-44], [5.21879960396506e-90, 1.0659574468085128e-40, 5.224361215265557e-90], [6.323597618525216e-73, 5.218750004771451e-89, 5.218750000004658e-89], [5.218750000000028e-88, 5.218750000000028e-88, 6.10976185251875e-74], [5.676668983476423e-88, 1.4735294117647084e-24, 5.218750000000025e-87], [6.423076923652442e-68, 5.676172991118479e-87, 5.6761729434041955e-87], [5.22106060626359e-86, 5.169958935339038e-86, 6.592105263163665e-65], [5.164997571255331e-85, 1.4468825910931199e-25, 5.164997571255331e-85], [5.164953799463096e-84, 5.164953317685938e-84, 0.0002783333333333334], [5.673366424512393e-84, 1.2846153846153867e-24, 5.164948501322556e-83],

In [111]:
count = Counter()

for q in qtable[:50]:
    m = 0
    mi = 0
    for i, v in enumerate(q):
        if v > m:
            m = v
            mi = i
            
    count[mi]+=1
    
print(count)

Counter({0: 22, 1: 18, 2: 10})
